# Introduction to Databases

In this chapter, we won't work directly with SQL but will more talk about the history of computerized databases.

To experiment with the data used for the examples, I downloaded and installed MySQL, and loaded the sakila examplease database.

## Definition of Database

A database is a set of related information. In a database, we would like to
* find information quickly
* find information easily
* be able to update it easily


### Types of Database Systems

1. Nonrelational Database Systems: 